### classify stackoverflow posts by convoluting over its individual word embeddings

**TODO**
 - use a different scoring function so that wrong guesses that are close to the correct values (think "python", "python-2") are penalized less than guesses that are wildly wrong.


In [18]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalMaxPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

In [3]:
SEED=np.random.randint(1,1000)
SEED

103

In [4]:
np.random.seed(seed=SEED)

In [5]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [40]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=1 # it starts overfitting after 1 epochs

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [10]:
[inverse_word_index[idx] for idx in sequences[0]][:20]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for']

In [11]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [12]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [13]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [14]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [15]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((32000, 1000), (32000, 1209))

In [16]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [81]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.8))
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam')

model.fit(X_train, Y_train,
                    batch_size=BATCH_SIZE,
                    epochs=NUM_EPOCHS,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 25600 samples, validate on 6400 samples
Epoch 1/1
 4096/25600 [===>..........................] - ETA: 59s - loss: 14.7858

KeyboardInterrupt: 

In [ ]:
def get_predicted_indices_by_tag_doc_fraction(
    tag_probabilities_index,
    predicted_tag_probabilities,
    multiplier=1.0,
    relative_difference_threshold=None):
    """
    
    (if predicted_index is not in the index, assume the average over all tags
    """
       
    output = []

    # this can be put outside to save time
    sum_over_all_tags = sum([value for _,value in tag_probabilities_index.items()])

    # no need for float casting in python 3
    mean_over_all_tags = sum_over_all_tags / len(tag_probabilities_index)

    for i,prob in enumerate(predicted_tag_probabilities.ravel()):

        avg_prob = tag_probabilities_index.get(i)
        
        relative_difference = (prob-avg_prob)/avg_prob
        
        if avg_prob is None:
            if prob > mean_over_all_tags:
                output.append(1)
            else:
                output.append(0)
        elif relative_difference_threshold is None:
            if prob > avg_prob * multiplier:
                output.append(1)
            else:
                output.append(0)
        else:   
            if prob > avg_prob and relative_difference > relative_difference_threshold:
                output.append(1)
            else:
                output.append(0)

    return np.array(output).reshape(1,-1)

In [ ]:
def get_top_k_predicted_indices(
    predicted_tag_probabilities,
    k=5):
    """
    
    (if predicted_index is not in the index, assume the average over all tags
    """
    
    
    tags_probs = [ (tag,prob) for tag,prob in enumerate(predicted_tag_probabilities.ravel())]
   
    sorted_tags_probs = sorted(tags_probs,key=lambda tpl:tpl[1],reverse=True)
       
    tags_ordered = [tag for tag,_ in sorted_tags_probs][:k]
    
    predicted_label_indices = np.zeros(len(tags_probs),dtype='int64')
       
    predicted_label_indices[tags_ordered] = 1   
       
    predicted_label_indices = predicted_label_indices.reshape(1,-1)
    
    return predicted_label_indices

In [ ]:
def get_predicted_indices_by_threshold(predicted_tag_probabilities,
                                   threshold=0.08):
    
    predicted_label_indices= np.zeros(len(predicted_tag_probabilities),dtype='int64')
    
    predicted_label_indices[predicted_tag_probabilities > threshold] = 1

    return predicted_label_indices

In [44]:
def show_sample_predictions(
    strategy,
    actual_label_indices,
    predicted_probabilities):
    
    supported_strategies = [
        'static_threshold',
        'document_fraction',
        'raw_difference',
        'relative_difference',
        'top_k'
    ]
    
    assert strategy in supported_strategies
    
    if strategy == 'static_threshold':
        predicted_label_indices = tags_helper.get_predicted_indices_by_threshold(
        predicted_tag_probabilities,
        threshold=threshold)
    

In [76]:
# sample result for a couple of test cases
num_test_cases = 10
threshold=0.15
multiplier=150.0

relative_difference_threshold = 175.0

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1)).ravel()
       
#     predicted_label_indices = tags_helper.get_predicted_indices_by_tag_doc_fraction(
#         tag_probabilities_index,
#         predicted_tag_probabilities,
#         multiplier=multiplier,
#     relative_difference_threshold=relative_difference_threshold)
        
#     predicted_label_indices = tags_helper.get_predicted_indices_by_threshold(
#         predicted_tag_probabilities,
#         threshold=threshold)
            
    predicted_label_indices = tags_helper.get_top_k_predicted_indices(
        predicted_tag_probabilities,
        k=5)
    
    actual_labels = lb.inverse_transform(actual_label_indices)
    
    predicted_labels = lb.inverse_transform(predicted_label_indices.reshape(1,-1))
    
    scores = predicted_tag_probabilities[predicted_label_indices.nonzero()[0]]
    
    predicted_labels_and_scores = list(zip(predicted_labels[0],scores))
    
    print(actual_labels)
    print( sorted(predicted_labels_and_scores,key=lambda x:x[1],reverse=True) )
    print('\n')


[('javascript', 'jquery', 'php', 'video')]
[('android', 0.03326172), ('c#', 0.03326172), ('java', 0.03326172), ('javascript', 0.03326172), ('python', 0.03326172)]


[('css',)]
[('android', 0.040229611), ('c#', 0.040229611), ('java', 0.040229611), ('javascript', 0.040229611), ('python', 0.040229611)]


[('caching', 'hibernate', 'spring-mvc')]
[('android', 0.024881983), ('c#', 0.024881983), ('java', 0.024881983), ('javascript', 0.024881983), ('php', 0.024881983)]


[('asp.net-mvc-5', 'c#', 'dependency-injection')]
[('android', 0.046581853), ('c#', 0.046581853), ('java', 0.046581853), ('javascript', 0.046581853), ('python', 0.046581853)]


[('mysql', 'php', 'sql')]
[('android', 0.026125433), ('c#', 0.026125433), ('java', 0.026125433), ('javascript', 0.026125433), ('python', 0.026125433)]


[('java', 'methods', 'mocking', 'replace')]
[('android', 0.041067682), ('c#', 0.041067682), ('java', 0.041067682), ('javascript', 0.041067682), ('python', 0.041067682)]


[()]
[('android', 0.033638801),

In [78]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [79]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

# doc_fraction
# Y_pred_calculated_lst = [tags_helper.get_predicted_indices_by_tag_doc_fraction(
#     tag_probabilities_index,
#     y_pred,
#     multiplier=multiplier,
#     relative_difference_threshold=relative_difference_threshold) for y_pred in np.vstack(Y_pred_lst)]

# static threshold
# Y_pred_calculated_lst = [tags_helper.get_predicted_indices_by_threshold(
#     y_pred,
#     threshold=threshold) for y_pred in np.vstack(Y_pred_lst)]

# top k
Y_pred_calculated_lst = [tags_helper.get_top_k_predicted_indices(
    y_pred,
    k=5) for y_pred in np.vstack(Y_pred_lst)]


Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

micro_f1 = metrics_helper.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))

micro-averaged F1 score (validation set) is 0.10689441126736444
micro-averaged precision score (validation set) is 0.07803125
micro-averaged recall score (validation set) is 0.16964467694816224
